not working yet 

In [116]:
import numpy as np
import pandas as pd
from SALib.sample import saltelli
from SALib.analyze import sobol

In [117]:
def mu_eq(mu_max, c_glucose, Ks, c_biomass, X_max):
    # -- MONOD / insert: mu_max, c_glucose, Ks
    #mu = mu_max * c_glucose / (c_glucose + Ks)
    # -- LOGISTIC / insert: mu_max, c_biomass, X_max
    #mu = mu_max * (1 - (c_biomass/ X_max)) 
    # -- MONOD + LOGISTIC / insert: mu_max, c_glucose, Ks, c_biomass, X_max
    mu = mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max))
    # -- MONOD + LOGISTIC + INHIBITION / insert: mu_max, c_glucose, Ks, Ki, c_biomass, X_max
    #mu = mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max))
    return mu


def qs_eq(Yxs, f_glucose, V):
    # -- MONOD / insert: qs_max, c_glucose, Ks_qs
    #qs = qs_max * c_glucose / (Ks_qs + c_glucose)
    # -- MONOD + NON COMPETITIVE INHIBITION / insert: qs_max, c_glucose, Ks_qs, Ki, glu_met
    #s = qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met))
    # -- YIELD / insert: Yxs, f_glucose, V
    qs = 1/Yxs * f_glucose / V #NOT SURE IF CORRECT because g_S/g_X but not per h
    # -- MONOD + METABOLIZED GLU / insert: qs_max, c_glucose, Ks_qs, glu_met, lag
    #qs = qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))
    return qs

In [118]:

def dXdt(mu, c_biomass, f_glucose, V):
    # -- BATCH + mu / insert: mu, c_biomass
    #dXdt = mu * c_biomass
    # -- FEDBATCH + mu / insert: mu, c_biomass, f_glucose, V
    dXdt = mu * c_biomass - c_biomass * f_glucose / V 
    # -- FEDBATCH + Yield / insert: qs, Yxs, c_biomass, f_glucose, V
    # dXdt = qs * Yxs * c_biomass - c_biomass * f_glucose / V
    return dXdt

def dSdt(f_glucose, V, c_glu_feed, c_glucose, qs, c_biomass, m_s):
    # -- BATCH / insert: qs, c_biomass
    # dSdt = -qs * c_biomass
    # -- FEDBATCH / insert: f_glucose, V, c_glu_feed, c_glucose, c_biomass, qs, c_biomass
    # dSdt = ((f_glucose / V) * (c_glu_feed - c_glucose)) - qs * c_biomass
    # -- FEDBATCH + MAINTENANCE / insert: f_glucose, V, c_glu_feed, c_glucose, qs, c_biomass, m_s
    dSdt = ((f_glucose / V) * (c_glu_feed - c_glucose)) - qs * c_biomass - m_s * c_biomass  
    return dSdt

In [119]:
def model(param):
    """
    Simulates the fermentation process based on the provided parameters.
    Args:
        params (dict): Dictionary containing the model parameters.
        t0 (float): Initial time.
        t_end (float): End time.
        dt (float): Time step size.
    Returns:
        time (array): Array of time values.
        biomass (array): Array of biomass concentrations.
        substrate (array): Array of substrate concentrations.
    """
    # Simulation settings
    t0 = 0
    t_end = 46.1
    dt = 1/60
    num_steps = int((t_end - t0) / dt) + 1 # Number of time steps
    print(num_steps)

    # Extract experimental data
    df_exp = pd.read_csv('data/data_combined.csv')
    F_glu = df_exp['Glucose feed [L/min]']*60 # [L/h]
    F_in = df_exp['Feed total [L/min]']

    mu_max = 0.2
    X_max = 23.1
    Ks = 7
    Ks_qs = 0.035
    Ki = 7.15
    Yxs = 0.48
    qs_max = 0.57
    m_s = 0.1
    lag = 0.0057 
    X0 = 0.5
    S0 = 5.5
    V0 = 1.0
    c_glu_feed = 350 

    # Arrays to store results
    time = np.linspace(t0, t_end, num_steps)
    biomass = np.zeros(num_steps)
    substrate = np.zeros(num_steps)
    volume = np.zeros(num_steps)
    S_met = np.zeros(num_steps)
    dS_dt = np.zeros(num_steps)

    # Set initial values
    biomass[0] = X0
    substrate[0] = S0
    volume[0] = V0

    # Simulation loop
    for i in range(1, num_steps):
        c_glucose = substrate[i-1]
        c_biomass = biomass[i-1]
        V = volume[i-1]
        glu_met = S_met[i-1]
        f_glucose = F_glu[i]
        
        
        # since the glucose concentration can't be negative, it is set to zero
        if c_glucose < 0:
            c_glucose = 0

        # Update growth and glucose uptake rate
        mu = mu_eq(mu_max, c_glucose, Ks, c_biomass, X_max)
        qs = qs_eq(Yxs, f_glucose, V)
        S_met[i] = qs * c_biomass * V

        # Update biomass and substrate concentrations
        dX_dt = dXdt(mu, c_biomass, f_glucose, V)
        dS_dt[i] = dSdt(f_glucose, V, c_glu_feed, c_glucose, qs, c_biomass, m_s)
        dV_dt = F_in[i] - (0.4/num_steps) # not complete -- somehow add base and acid to it + include samples + evaporation
        biomass[i] = c_biomass + dX_dt * dt
        substrate[i] = c_glucose + dS_dt[i] * dt
        volume[i] = V + dV_dt

    return time, biomass, substrate, volume, dS_dt


In [120]:

# Define the parameter bounds
problem = {
    'num_vars': 11,
    'names': ['mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'X0', 'c_glu_feed'],
    'bounds': [[0.1, 0.5], [1.0, 10.0], [0.01, 1.0], [0.01, 1.0], [0.01, 1.0], [0.1, 0.5], [0.1, 1.0], [0.01, 1.0], [0.0, 1.0], [0.1, 1.0], [100, 500]]
}


In [121]:
# Generate the parameter samples using the Saltelli sampling scheme
param_samples = saltelli.sample(problem, 4)

/var/folders/nf/38rh41fn3szgbs5wpsdwjvk00000gn/T/ipykernel_16531/3128660257.py:2: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_samples = saltelli.sample(problem, 4)


In [122]:

# Run the model for each parameter sample and collect the output

outputs = np.array([model(param) for param in param_samples])

2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767
2767


In [123]:

# Perform the Sobol sensitivity analysis
Si = sobol.analyze(problem, outputs)


ValueError: could not broadcast input array from shape (4,5,2767) into shape (55340,)

In [ ]:



# Print the first-order and total sensitivity indices
print("First-order indices:")
for i, name in enumerate(problem['names']):
    print("{}: {:.3f}".format(name, Si['S1'][i]))

print("\nTotal indices:")
for i, name in enumerate(problem['names']):
    print

First-order indices:


NameError: name 'Si' is not defined